# 百度地图地铁数据处理


In [123]:
import pandas as pd
import numpy as np
import json

In [111]:
import sys
sys.path.append("../../..")
from secure.db_account import SubwayPrd
from k_libs.db_query import DBOperate

DBO = DBOperate(SubwayPrd)

## 字段表


In [3]:
# 线路字段
line_fields = [
    {'raw': 'lid',
    'processed': 'line_name_full',
    'info': '线路全名'},
    {'raw': 'lb',
    'processed': 'line_name',
    'info': '线路简称'},
    {'raw': 'n',
    'processed': 'num_stations',
    'info': '线路站点数'},
    {'raw': 'loop',
    'processed': 'is_loop',
    'info': '是否环线'},
    {'raw': 'lbx',
    'processed': 'label_x',
    'info': '线路标签X坐标'},
    {'raw': 'lby',
    'processed': 'label_y',
    'info': '线路标签Y坐标'},
    {'raw': 'lbr',
    'processed': 'label_rotation',
    'info': '线路标签旋转角度'},
    {'raw': 'lc',
    'processed': 'line_color',
    'info': '线路颜色'},
    {'raw': 'uid',
    'processed': 'line_uid',
    'info': '线路唯一ID'},
    {'raw': 'uid2',
    'processed': 'line_uid2',
    'info': '线路备用唯一ID'}
]

In [4]:
# 站点字段
station_fields = [
    {'raw': 'sid',
    'processed': 'station_name',
    'info': '站点名称'},
    {'raw': 'lb',
    'processed': 'station_label',
    'info': '站点标签'},
    {'raw': 'x',
    'processed': 'x',
    'info': '站点X坐标'},
    {'raw': 'y',
    'processed': 'y',
    'info': '站点Y坐标'},
    {'raw': 'rx',
    'processed': 'label_offset_x',
    'info': '站点标签X偏移'},
    {'raw': 'ry',
    'processed': 'label_offset_y',
    'info': '站点标签Y偏移'},
    {'raw': 'st',
    'processed': 'is_station',
    'info': '是否为车站'},
    {'raw': 'ex',
    'processed': 'is_exchange',
    'info': '是否换乘点'},
    {'raw': 'iu',
    'processed': 'is_use',
    'info': '是否在用,true表示该站已开通并运营'},
    {'raw': 'rc',
    'processed': 'is_rail_construction',
    'info': '是否为规划/在建站点'},
    {'raw': 'slb',
    'processed': 'show_label',
    'info': '是否显示标签'},
    {'raw': 'ln',
    'processed': 'lines',
    'info': '所属线路'},
    {'raw': 'uid',
    'processed': 'station_uid',
    'info': '站点唯一ID'},
    {'raw': 'px',
    'processed': 'proj_x',
    'info': '站点投影坐标X'},
    {'raw': 'py',
    'processed': 'proj_y',
    'info': '站点投影坐标Y'}
    ]


## 原始数据
ODS 层（Operational Data Store，操作数据存储层）

存放业务系统的原始数据，基本不做加工。

特点：与业务库字段保持一致，保证数据的完整性与可追溯性。

作用：承接源系统，作为数据仓库的“原材料”。

In [5]:
sql = """SELECT * FROM ods_subway_baidu WHERE crawler_id=(SELECT MAX(crawler_id) FROM ods_subway_baidu);"""
df_raw = DBO.read_sql(sql)

In [7]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15923 entries, 0 to 15922
Data columns (total 33 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   line_lid      15923 non-null  object
 1   line_lb       15923 non-null  object
 2   line_slb      15923 non-null  object
 3   line_n        15923 non-null  object
 4   line_loop     15923 non-null  object
 5   line_lbx      15923 non-null  object
 6   line_lby      15923 non-null  object
 7   line_lbr      15923 non-null  object
 8   line_lc       15923 non-null  object
 9   line_uid      15923 non-null  object
 10  line_uid2     15923 non-null  object
 11  st_sid        15923 non-null  object
 12  st_lb         15923 non-null  object
 13  st_x          15923 non-null  object
 14  st_y          15923 non-null  object
 15  st_rx         15923 non-null  object
 16  st_ry         15923 non-null  object
 17  st_st         15923 non-null  object
 18  st_ex         15923 non-null  object
 19  st_i

In [6]:
df_raw

,line_lid,line_lb,line_slb,line_n,line_loop,line_lbx,line_lby,line_lbr,line_lc,line_uid,...,st_int,st_uid,st_px,st_py,city_id,city_name,city_name_e,crawler_id,crawler_date,uid
0,地铁1号线八通线,1号线八通线,False,23,False,-498.9,139.1,0,0xc03935,bce557d6f7fadd4ea5da39b7,...,3,291c5802f26a751cbca240d9,12935140.04,4825694.5,131,北京,beijing,250928,2025-09-28,1312509280
1,地铁1号线八通线,1号线八通线,False,23,False,-498.9,139.1,0,0xc03935,bce557d6f7fadd4ea5da39b7,...,3,ad28546df35285eb851541d9,12937624.6,4825645.68,131,北京,beijing,250928,2025-09-28,1312509281
2,地铁1号线八通线,1号线八通线,False,23,False,-498.9,139.1,0,0xc03935,bce557d6f7fadd4ea5da39b7,...,3,a047555503a5bc5cbc4842d9,12940185.58,4825661.64,131,北京,beijing,250928,2025-09-28,1312509282
3,地铁1号线八通线,1号线八通线,False,23,False,-498.9,139.1,0,0xc03935,bce557d6f7fadd4ea5da39b7,...,3,a27fa5a23a128501177643d9,12942086.4,4825707.12,131,北京,beijing,250928,2025-09-28,1312509283
4,地铁1号线八通线,1号线八通线,False,23,False,-498.9,139.1,0,0xc03935,bce557d6f7fadd4ea5da39b7,...,3,53889c15034f2e3f58bbbcde,12944444.84,4825755.15,131,北京,beijing,250928,2025-09-28,1312509284
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15918,ML4,ML4,False,16,False,-110,1600,0,0x77b838,a1bf283eb66e3574c9c3dbb7,...,2,None,None,None,51314,马德里,madeli,250928,2025-09-28,51314250928445
15919,ML4,ML4,False,16,False,-110,1600,0,0x77b838,a1bf283eb66e3574c9c3dbb7,...,2,None,None,None,51314,马德里,madeli,250928,2025-09-28,51314250928446
15920,ML4,ML4,False,16,False,-110,1600,0,0x77b838,a1bf283eb66e3574c9c3dbb7,...,2,5660ab357cf66173901fc9b7,-417722.43,4872240.68,51314,马德里,madeli,250928,2025-09-28,51314250928447
15921,ML4,ML4,False,16,False,-110,1600,0,0x77b838,a1bf283eb66e3574c9c3dbb7,...,2,d79642c2e73da856eedfcab7,-417875.2,4871682.71,51314,马德里,madeli,250928,2025-09-28,51314250928448


## DWD 
DWD 层（Data Warehouse Detail，明细数据层）  
在 ODS 的基础上进行清洗、规范化，保留明细粒度的数据。  
特点：字段标准化（统一命名、数据类型），做一些维度退化或拆分。  
作用：保证数据“可用”，是最常被下游加工使用的一层。  

In [112]:
def dwd_subway_bd_to_sql(df_raw):
    """
    将百度地铁数据处理为DWD层数据，并存入数据库
    """

    # 对字段重命名
    df_dwd = df_raw.copy()
    df_dwd = df_dwd.rename(columns={
        'line_lid': 'line_name_full',
        'line_lb': 'line_name',
        'line_slb': 'line_show_label',
        'line_loop': 'line_is_loop',
        'line_lbx': 'line_label_x',
        'line_lby': 'line_label_y',
        'line_lbr': 'line_label_rotation',
        'line_lc': 'line_color',
        'st_sid': 'st_name',
        'st_lb': 'st_label',
        'st_rx': 'st_label_offset_x',
        'st_ry': 'st_label_offset_y',
        'st_st': 'st_is_station',
        'st_ex': 'st_is_exchange',
        'st_iu': 'st_is_use',
        'st_rc': 'st_is_rail_construction',
        'st_slb': 'st_show_label',
        'st_ln': 'st_lines',
        'st_px': 'st_proj_x',
        'st_py': 'st_proj_y'}
    )
    # 字段排序
    field_order = [
        'city_id', 'city_name', 'city_name_e',
        # 线路信息
        'line_name_full', 'line_name', 'line_show_label', 'line_n', 'line_is_loop',
        'line_label_x', 'line_label_y', 'line_label_rotation', 'line_color',
        'line_uid', 'line_uid2',
        # 站点信息
        'st_name', 'st_label', 'st_x', 'st_y', 'st_label_offset_x', 'st_label_offset_y',
        'st_is_station', 'st_is_exchange', 'st_is_use', 'st_is_rail_construction',
        'st_show_label', 'st_lines', 'st_int', 'st_uid', 'st_proj_x', 'st_proj_y',
        # 数据采集信息
        'crawler_id', 'crawler_date', 'uid'
    ]
    df_dwd = df_dwd[field_order]
    # 对存在false,true的字段进行处理
    bool_fields = [
        'line_show_label', 'line_is_loop', 'st_is_station', 'st_is_exchange', 'st_is_use', 'st_is_rail_construction', 'st_show_label'
    ]
    df_dwd = df_dwd.copy()
    for field in bool_fields:
        df_dwd[field] = df_dwd[field].map({'True': 1, 'False': 0})
    # 颜色修改为16进制
    df_dwd['line_color'] = df_dwd['line_color'].apply(lambda x: f"#{x[2:]}" if not x.startswith('#') else x)
    
    # 保存到数据库
    DBO.df_to_sql(df_dwd, "dwd_subway_baidu", if_exists="replace")
    print("Data saved to dwd_subway_baidu table.")
    return df_dwd

dwd_subway = dwd_subway_bd_to_sql(df_raw)
dwd_subway.info()

Data saved to dwd_subway_baidu table.
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15923 entries, 0 to 15922
Data columns (total 33 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   city_id                  15923 non-null  object 
 1   city_name                15923 non-null  object 
 2   city_name_e              15923 non-null  object 
 3   line_name_full           15923 non-null  object 
 4   line_name                15923 non-null  object 
 5   line_show_label          15923 non-null  int64  
 6   line_n                   15923 non-null  object 
 7   line_is_loop             15923 non-null  int64  
 8   line_label_x             15923 non-null  object 
 9   line_label_y             15923 non-null  object 
 10  line_label_rotation      15923 non-null  object 
 11  line_color               15923 non-null  object 
 12  line_uid                 15923 non-null  object 
 13  line_uid2                15923 non-nul

## DWS
DWS 层（Data Warehouse Summary，汇总数据层）

在 DWD 基础上，按业务主题和常用维度做聚合统计。

特点：以主题域为核心（如用户、订单、交易），形成宽表或统计指标。

作用：减少重复计算，支撑公共分析需求。

1. 城市表 dws_subway_bd_city
2. 线路表 dws_subway_bd_line
3. 车站表 dws_subway_bd_st

### 城市表

In [20]:
with open("city_info.json", "r", encoding="utf-8") as f:
    city_info = json.load(f)
city_info_df = pd.DataFrame.from_dict(city_info['cities'])
city_info_df

,city_name,english_pinyin_name,province_chinese,province_pinyin_english,country_chinese,country_english
0,北京,Beijing,北京市,Beijing Shi,中国,China
1,上海,Shanghai,上海市,Shanghai Shi,中国,China
2,广州,Guangzhou,广东省,Guangdong Sheng,中国,China
3,深圳,Shenzhen,广东省,Guangdong Sheng,中国,China
4,重庆,Chongqing,重庆市,Chongqing Shi,中国,China
...,...,...,...,...,...,...
74,莫斯科,Moscow,莫斯科州,Moscow Oblast,俄罗斯,Russia
75,鹿特丹,Rotterdam,南荷兰省,South Holland Province,荷兰,Netherlands
76,伊斯坦布尔,Istanbul,伊斯坦布尔省,Istanbul Province,土耳其,Turkey
77,巴塞罗那,Barcelona,加泰罗尼亚自治区,Catalonia Autonomous Community,西班牙,Spain


In [22]:
df_city = dwd_subway[['city_id', 'city_name']].drop_duplicates().reset_index(drop=True)
df_city['city_order'] = df_city.index + 1
df_city

,city_id,city_name,city_order
0,131,北京,1
1,289,上海,2
2,257,广州,3
3,340,深圳,4
4,132,重庆,5
...,...,...,...
74,65531,莫斯科,75
75,52390,鹿特丹,76
76,48552,伊斯坦布尔,77
77,51271,巴塞罗那,78


In [26]:
df_city = df_city.merge(city_info_df, how='left', on='city_name')
df_city

,city_id,city_name,city_order,english_pinyin_name,province_chinese,province_pinyin_english,country_chinese,country_english
0,131,北京,1,Beijing,北京市,Beijing Shi,中国,China
1,289,上海,2,Shanghai,上海市,Shanghai Shi,中国,China
2,257,广州,3,Guangzhou,广东省,Guangdong Sheng,中国,China
3,340,深圳,4,Shenzhen,广东省,Guangdong Sheng,中国,China
4,132,重庆,5,Chongqing,重庆市,Chongqing Shi,中国,China
...,...,...,...,...,...,...,...,...
74,65531,莫斯科,75,Moscow,莫斯科州,Moscow Oblast,俄罗斯,Russia
75,52390,鹿特丹,76,Rotterdam,南荷兰省,South Holland Province,荷兰,Netherlands
76,48552,伊斯坦布尔,77,Istanbul,伊斯坦布尔省,Istanbul Province,土耳其,Turkey
77,51271,巴塞罗那,78,Barcelona,加泰罗尼亚自治区,Catalonia Autonomous Community,西班牙,Spain


In [229]:
dwd_subway_bd_city = df_city

### 线路表
* line_uid有空值，不可用
* line_name_full 暂无重复值，可以用
* line_name 有重复值，需处理

#### 支线名称清洗

In [183]:
df_line = dwd_subway[['city_id', 'city_name', 'line_name_full', 'line_name', 'line_is_loop', 'line_label_x', 'line_label_y', 'line_label_rotation', 'line_color']].drop_duplicates().reset_index(drop=True)


In [184]:
# 按city_name, line_name统计line_name出现次数
df_line['line_name_count'] = df_line.groupby(['city_name', 'line_name'])['line_name'].transform('count')
df_line[df_line['line_name_count'] > 1]

,city_id,city_name,line_name_full,line_name,line_is_loop,line_label_x,line_label_y,line_label_rotation,line_color,line_name_count
55,257,广州,地铁3号线(海傍-天河客运站),3号线,0,230,-300,0,#EEB56E,2
56,257,广州,地铁3号线(体育西路-机场北(2号航站楼)),3号线,0,-60,-530,0,#EEB56E,2
566,65531,莫斯科,Line 4(昆采沃-亚历山大公园),Line 4,0,-457,-170,0,#54c6f2,2
567,65531,莫斯科,Line 4(国际-基辅),Line 4,0,-380,-80,0,#54c6f2,2
585,48552,伊斯坦布尔,M2,M2,0,-75.6,-691,0,#0fa249,2
586,48552,伊斯坦布尔,M2S,M2,0,-75.6,-691,0,#0fa249,2
631,51314,马德里,ML42,ML4,0,-110,1600,0,#77b838,2
632,51314,马德里,ML4,ML4,0,-110,1600,0,#77b838,2


In [185]:
# 按city_name, line_name_full统计line_name出现次数
df_line['line_name_full_count'] = df_line.groupby(['city_name', 'line_name_full'])['line_name_full'].transform('count')
df_line[df_line['line_name_full_count'] > 1]
# 注意，当下表不为空时，说明line_name_full也有重复，需要进一步处理

,city_id,city_name,line_name_full,line_name,line_is_loop,line_label_x,line_label_y,line_label_rotation,line_color,line_name_count,line_name_full_count


In [186]:
# 添加line_name_branch，当line_name_count>1时，使用line_name_full
df_line['line_name_branch'] = df_line['line_name']
df_line.loc[df_line['line_name_count'] > 1, 'line_name_branch'] = df_line['line_name_full']
# 删除line_name_full中的“地铁”
df_line['line_name_branch'] = df_line['line_name_branch'].str.replace('地铁', '', regex=False)
df_line

,city_id,city_name,line_name_full,line_name,line_is_loop,line_label_x,line_label_y,line_label_rotation,line_color,line_name_count,line_name_full_count,line_name_branch
0,131,北京,地铁1号线八通线,1号线八通线,0,-498.9,139.1,0,#c03935,1,1,1号线八通线
1,131,北京,地铁2号线,2号线,1,-175.3,-4.9,0,#005f98,1,1,2号线
2,131,北京,地铁3号线,3号线,0,450,-55,0,#942413,1,1,3号线
3,131,北京,地铁4号线大兴线,4号线大兴线,0,-499.3,-224,0,#008e9c,1,1,4号线大兴线
4,131,北京,地铁5号线,5号线,0,18,-353.9,0,#a6217f,1,1,5号线
...,...,...,...,...,...,...,...,...,...,...,...,...
628,51314,马德里,ML1,ML1,0,-70,-960,0,#5086c2,1,1,ML1
629,51314,马德里,ML2,ML2,0,-780,-280,0,#ea4d81,1,1,ML2
630,51314,马德里,ML3,ML3,0,-1800,295.2,0,#ff6642,1,1,ML3
631,51314,马德里,ML42,ML4,0,-110,1600,0,#77b838,2,1,ML42


In [187]:
# 检查广州三号线
df_line[df_line['city_name'] == '广州']

,city_id,city_name,line_name_full,line_name,line_is_loop,line_label_x,line_label_y,line_label_rotation,line_color,line_name_count,line_name_full_count,line_name_branch
53,257,广州,地铁1号线,1号线,0,-520,250,0,#ECD35E,1,1,1号线
54,257,广州,地铁2号线,2号线,0,-160,-480,0,#4288B4,1,1,2号线
55,257,广州,地铁3号线(海傍-天河客运站),3号线,0,230,-300,0,#EEB56E,2,1,3号线(海傍-天河客运站)
56,257,广州,地铁3号线(体育西路-机场北(2号航站楼)),3号线,0,-60,-530,0,#EEB56E,2,1,3号线(体育西路-机场北(2号航站楼))
57,257,广州,地铁4号线,4号线,0,1160,830,0,#4BA37A,1,1,4号线
58,257,广州,地铁5号线,5号线,0,880,300,0,#D34F6C,1,1,5号线
59,257,广州,地铁6号线,6号线,0,-790,-345,0,#A36E8C,1,1,6号线
60,257,广州,地铁7号线,7号线,0,560,-580,0,#8FC322,1,1,7号线
61,257,广州,地铁8号线,8号线,0,-565,-610,0,#00A1CC,1,1,8号线
62,257,广州,地铁9号线,9号线,0,-940,-870,0,#8DCFAA,1,1,9号线


In [188]:
# 按城市，线路顺序添加line_order
df_line['line_order'] = df_line.groupby('city_id').cumcount() + 1
df_line


,city_id,city_name,line_name_full,line_name,line_is_loop,line_label_x,line_label_y,line_label_rotation,line_color,line_name_count,line_name_full_count,line_name_branch,line_order
0,131,北京,地铁1号线八通线,1号线八通线,0,-498.9,139.1,0,#c03935,1,1,1号线八通线,1
1,131,北京,地铁2号线,2号线,1,-175.3,-4.9,0,#005f98,1,1,2号线,2
2,131,北京,地铁3号线,3号线,0,450,-55,0,#942413,1,1,3号线,3
3,131,北京,地铁4号线大兴线,4号线大兴线,0,-499.3,-224,0,#008e9c,1,1,4号线大兴线,4
4,131,北京,地铁5号线,5号线,0,18,-353.9,0,#a6217f,1,1,5号线,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
628,51314,马德里,ML1,ML1,0,-70,-960,0,#5086c2,1,1,ML1,13
629,51314,马德里,ML2,ML2,0,-780,-280,0,#ea4d81,1,1,ML2,14
630,51314,马德里,ML3,ML3,0,-1800,295.2,0,#ff6642,1,1,ML3,15
631,51314,马德里,ML42,ML4,0,-110,1600,0,#77b838,2,1,ML42,16


#### 主线名称清洗
1. 计算line_name与上一行相同字数，作为same_prefix_len列
2. 筛选出same_prefix_len>2的行，以及每一的上一行
3. 将['city_id', 'city_name', 'line_name_full', 'line_name', 'line_name_branch']保存为新的df
4. 对比subway_line_fixed.json文件，将新增的手工添加"line_name_main"字段，复制到subway_line_fixed.json文件最后。

In [ ]:
df_line_main = df_line.copy()
# 按城市，统计line_name与上一行line_name相同的字数，从首字开始统计，如首字不同，直接记为0
df_line_main['same_prefix_len'] = 0
for city in df_line_main['city_name'].unique():
    city_mask = df_line_main[df_line_main['city_name'] == city]
    prev_line_name = ""
    for idx, row in city_mask.iterrows():
        line_name = row['line_name']
        if line_name == prev_line_name:
            df_line_main.at[idx, 'same_prefix_len'] = len(line_name)
        else:
            # 计算与上一行相同的前缀长度
            common_length = 0
            for c1, c2 in zip(line_name, prev_line_name):
                if c1 == c2:
                    common_length += 1
                else:
                    break
            df_line_main.at[idx, 'same_prefix_len'] = common_length
        prev_line_name = line_name
df_line_main

,city_id,city_name,line_name_full,line_name,line_is_loop,line_label_x,line_label_y,line_label_rotation,line_color,line_name_count,line_name_full_count,line_name_branch,line_order,same_prefix_len
0,131,北京,地铁1号线八通线,1号线八通线,0,-498.9,139.1,0,#c03935,1,1,1号线八通线,1,0
1,131,北京,地铁2号线,2号线,1,-175.3,-4.9,0,#005f98,1,1,2号线,2,0
2,131,北京,地铁3号线,3号线,0,450,-55,0,#942413,1,1,3号线,3,0
3,131,北京,地铁4号线大兴线,4号线大兴线,0,-499.3,-224,0,#008e9c,1,1,4号线大兴线,4,0
4,131,北京,地铁5号线,5号线,0,18,-353.9,0,#a6217f,1,1,5号线,5,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
628,51314,马德里,ML1,ML1,0,-70,-960,0,#5086c2,1,1,ML1,13,0
629,51314,马德里,ML2,ML2,0,-780,-280,0,#ea4d81,1,1,ML2,14,2
630,51314,马德里,ML3,ML3,0,-1800,295.2,0,#ff6642,1,1,ML3,15,2
631,51314,马德里,ML42,ML4,0,-110,1600,0,#77b838,2,1,ML42,16,2


In [ ]:
# 筛选出same_prefix_len>2的行，以及每一的上一行
df_line_main_filtered = pd.DataFrame()
for city in df_line_main['city_name'].unique():
    city_mask = df_line_main[df_line_main['city_name'] == city]
    indices = city_mask.index.tolist()
    for i in range(1, len(indices)):
        if city_mask.at[indices[i], 'same_prefix_len'] > 2:
            df_line_main_filtered = pd.concat([df_line_main_filtered, city_mask.loc[[indices[i-1], indices[i]]]])
df_line_main_filtered = df_line_main_filtered.drop_duplicates().reset_index(drop=True)
df_line_main_filtered

,city_id,city_name,line_name_full,line_name,line_is_loop,line_label_x,line_label_y,line_label_rotation,line_color,line_name_count,line_name_full_count,line_name_branch,line_order,same_prefix_len
0,131,北京,地铁17号线(十里河-嘉会湖),17号线(十里河-嘉会湖),0,450,700,0,#25989A,1,1,17号线(十里河-嘉会湖),17,1
1,131,北京,地铁17号线北段(未来科学城北-工人体育场),17号线(未来科学城北-工人体育场),0,108,-430,0,#25989A,1,1,17号线(未来科学城北-工人体育场),18,5
2,289,上海,地铁5号线(莘庄-奉贤新城),5号线,0,-499,987.3,0,#CC00CC,1,1,5号线,5,0
3,289,上海,地铁5号线支线(东川路-闵行开发区),5号线支线,0,-589,787.3,0,#CC00CC,1,1,5号线支线,6,3
4,289,上海,地铁10号线(基隆路-航中路),10号线(基隆路-航中路),0,-750,58.8,0,#C9A7D5,1,1,10号线(基隆路-航中路),11,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
164,51314,马德里,Line 10,Line 10,0,380,-1320,0,#0f6ab0,1,1,Line 10,10,5
165,51314,马德里,Line 11,Line 11,0,-700,670,0,#18a853,1,1,Line 11,11,6
166,51314,马德里,Line 12,Line 12,0,-1100,720,0,#a4991e,1,1,Line 12,12,6
167,51314,马德里,ML42,ML4,0,-110,1600,0,#77b838,2,1,ML42,16,2


In [ ]:
df_line_main_filtered_dict = df_line_main_filtered[['city_id', 'city_name', 'line_name_full', 'line_name', 'line_name_branch']].to_dict(orient='records')
df_line_main_filtered_dict

[{'city_id': '131',
  'city_name': '北京',
  'line_name_full': '地铁17号线(十里河-嘉会湖)',
  'line_name': '17号线(十里河-嘉会湖)',
  'line_name_branch': '17号线(十里河-嘉会湖)'},
 {'city_id': '131',
  'city_name': '北京',
  'line_name_full': '地铁17号线北段(未来科学城北-工人体育场)',
  'line_name': '17号线(未来科学城北-工人体育场)',
  'line_name_branch': '17号线(未来科学城北-工人体育场)'},
 {'city_id': '289',
  'city_name': '上海',
  'line_name_full': '地铁5号线(莘庄-奉贤新城)',
  'line_name': '5号线',
  'line_name_branch': '5号线'},
 {'city_id': '289',
  'city_name': '上海',
  'line_name_full': '地铁5号线支线(东川路-闵行开发区)',
  'line_name': '5号线支线',
  'line_name_branch': '5号线支线'},
 {'city_id': '289',
  'city_name': '上海',
  'line_name_full': '地铁10号线(基隆路-航中路)',
  'line_name': '10号线(基隆路-航中路)',
  'line_name_branch': '10号线(基隆路-航中路)'},
 {'city_id': '289',
  'city_name': '上海',
  'line_name_full': '地铁10号线(基隆路-虹桥火车站)',
  'line_name': '10号线(基隆路-虹桥火车站)',
  'line_name_branch': '10号线(基隆路-虹桥火车站)'},
 {'city_id': '289',
  'city_name': '上海',
  'line_name_full': '地铁11号线(嘉定北-迪士尼)',
  'line_name': '11号

In [227]:
with open("subway_line_fixed.json", "r", encoding="utf-8") as f:
    subway_line_fixed = json.load(f)
# 对比subway_line_fixed.json文件，打印df_line_main_filtered_dict中每项前4项不在subway_line_fixed中的行
fields = ['city_id', 'city_name', 'line_name_full', 'line_name']
fixed_set = set(
    tuple(line[field] for field in fields)
    for line in subway_line_fixed
)
for item in df_line_main_filtered_dict:
    key = tuple(item[field] for field in fields)
    if key not in fixed_set:
        print(item)
# 手动为上述打印的行添加"line_name_main"字段，复制到subway_line_fixed.json文件最后
# 保存subway_line_fixed.json文件

#### 合并线路数据

In [228]:
with open("subway_line_fixed.json", "r", encoding="utf-8") as f:
    subway_line_fixed = json.load(f)
subway_line_fixed_df = pd.DataFrame.from_dict(subway_line_fixed)
subway_line_fixed_df

,city_id,city_name,line_name_full,line_name,line_name_branch,line_name_main
0,131,北京,地铁17号线(十里河-嘉会湖),17号线(十里河-嘉会湖),17号线(十里河-嘉会湖),17号线
1,131,北京,地铁17号线北段(未来科学城北-工人体育场),17号线(未来科学城北-工人体育场),17号线(未来科学城北-工人体育场),17号线
2,289,上海,地铁5号线(莘庄-奉贤新城),5号线,5号线,5号线
3,289,上海,地铁5号线支线(东川路-闵行开发区),5号线支线,5号线支线,5号线
4,289,上海,地铁10号线(基隆路-航中路),10号线(基隆路-航中路),10号线(基隆路-航中路),10号线
...,...,...,...,...,...,...
164,51314,马德里,Line 10,Line 10,Line 10,Line 10
165,51314,马德里,Line 11,Line 11,Line 11,Line 11
166,51314,马德里,Line 12,Line 12,Line 12,Line 12
167,51314,马德里,ML42,ML4,ML42,ML4


In [233]:
dwd_subway_bd_line = df_line.merge(subway_line_fixed_df[['city_id', 'line_name_full', 'line_name_main']], how='left', on=['city_id', 'line_name_full']) 
# 将line_name_main中缺失值，填充为line_name_branch
dwd_subway_bd_line['line_name_main'] = dwd_subway_bd_line['line_name_main'].fillna(dwd_subway_bd_line['line_name_branch'])
dwd_subway_bd_line

,city_id,city_name,line_name_full,line_name,line_is_loop,line_label_x,line_label_y,line_label_rotation,line_color,line_name_count,line_name_full_count,line_name_branch,line_order,line_name_main
0,131,北京,地铁1号线八通线,1号线八通线,0,-498.9,139.1,0,#c03935,1,1,1号线八通线,1,1号线八通线
1,131,北京,地铁2号线,2号线,1,-175.3,-4.9,0,#005f98,1,1,2号线,2,2号线
2,131,北京,地铁3号线,3号线,0,450,-55,0,#942413,1,1,3号线,3,3号线
3,131,北京,地铁4号线大兴线,4号线大兴线,0,-499.3,-224,0,#008e9c,1,1,4号线大兴线,4,4号线大兴线
4,131,北京,地铁5号线,5号线,0,18,-353.9,0,#a6217f,1,1,5号线,5,5号线
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
628,51314,马德里,ML1,ML1,0,-70,-960,0,#5086c2,1,1,ML1,13,ML1
629,51314,马德里,ML2,ML2,0,-780,-280,0,#ea4d81,1,1,ML2,14,ML2
630,51314,马德里,ML3,ML3,0,-1800,295.2,0,#ff6642,1,1,ML3,15,ML3
631,51314,马德里,ML42,ML4,0,-110,1600,0,#77b838,2,1,ML42,16,ML4


### 车站表

In [234]:
def get_dwd_subway_st(df, df_line, is_all_st=True):
    """
    获取地铁站数据
    车站含虚拟车站和真实车站两种
    is_all_st: True-全量车站， False-真实车站
    df: dwd_subway数据
    df_line: dwd_subway_bd_line数据
    """
    df = df.copy() if is_all_st else df[df['st_is_station'] == 1].copy()
    print(f"Total stations: {len(df)}")
    # 连接查询线路顺序
    res_df = df.merge(df_line[['city_id', 'line_name_full', 'line_order', 'line_name_branch', 'line_name_main']], how='left', on=['city_id', 'line_name_full'])
    print(f"Total stations after merge line info: {len(res_df)}")
    # 当len(res_df != len(df))时，说明有部分站点的line_name_full在df_line中没有匹配上
    if len(res_df) != len(df):
        print(f"WARNNING!, Stations with unmatched line info: {len(res_df) - len(df)}")
    # 车站顺序
    res_df['st_order'] = res_df.groupby(['city_id', 'line_name_full']).cumcount() + 1
    # 车站id
    res_df['st_id_virtual'] = res_df.apply(lambda row: f"{row['city_id']}_{row['line_order']:03d}_{row['st_order']:03d}", axis=1)
    # 将st_name为空或字符串长度为0的站点st_name填充为st_id_virtual
    res_df['st_name'] = res_df['st_name'].replace('', np.nan)
    res_df['st_name'] = res_df['st_name'].fillna(res_df['st_id_virtual'])
    # 下一站id
    res_df['target_st_id_virtual'] = res_df.groupby(['city_id', 'line_name_full'])['st_id_virtual'].shift(-1)
    # 如果line_is_loop为1，则最后一站的下一站为第一站
    # 找出所有环线线路
    loop_keys = res_df.loc[res_df['line_is_loop'] == 1, ['city_id', 'line_name_full']].drop_duplicates()

    for _, row in loop_keys.iterrows():
        mask = (res_df['city_id'] == row['city_id']) & (res_df['line_name_full'] == row['line_name_full'])
        idx = res_df.loc[mask].index

        if len(idx) > 1:  # 至少要有两站才能构成环线
            first_idx = idx[0]
            last_idx = idx[-1]
            res_df.loc[last_idx, 'target_st_id_virtual'] = res_df.loc[first_idx, 'st_id_virtual']
    # 按target_st_id_virtual， 添加target_st_name, target_st_x, target_st_y
    res_df = res_df.merge(res_df[['st_id_virtual', 'st_name', 'st_x', 'st_y']].rename(columns={
        'st_id_virtual': 'target_st_id_virtual',
        'st_name': 'target_st_name',
        'st_x': 'target_st_x',
        'st_y': 'target_st_y'
    }), how='left', on='target_st_id_virtual')
    return res_df

#### 全量车站表
含虚拟车站，用于线条更柔和

In [235]:
df_st_all =  get_dwd_subway_st(dwd_subway, dwd_subway_bd_line, is_all_st=True)

Total stations: 15923
Total stations after merge line info: 15923


#### 真实车站表
真实存在的， 即st_is_station == 1

In [236]:
df_st_real =  get_dwd_subway_st(dwd_subway, dwd_subway_bd_line, is_all_st=False)


Total stations: 13291
Total stations after merge line info: 13291


## ADS
ADS 层（Application Data Store，应用数据层）

面向应用和报表的最终数据，通常是宽表、指标表。

特点：与具体应用、报表或接口一一对应。

作用：满足业务方“即取即用”，保证查询效率。

按城市统计指标


In [204]:
df_st_real

,city_id,city_name,city_name_e,line_name_full,line_name,line_show_label,line_n,line_is_loop,line_label_x,line_label_y,...,crawler_date,uid,line_order,line_name_branch,st_order,st_id_virtual,target_st_id_virtual,target_st_name,target_st_x,target_st_y
0,131,北京,beijing,地铁1号线八通线,1号线八通线,0,23,0,-498.9,139.1,...,2025-09-28,1312509280,1,1号线八通线,1,131_001_001,131_001_002,八角游乐园,-608.2,170.6
1,131,北京,beijing,地铁1号线八通线,1号线八通线,0,23,0,-498.9,139.1,...,2025-09-28,1312509281,1,1号线八通线,2,131_001_002,131_001_003,八宝山,-549.2,170.6
2,131,北京,beijing,地铁1号线八通线,1号线八通线,0,23,0,-498.9,139.1,...,2025-09-28,1312509282,1,1号线八通线,3,131_001_003,131_001_004,玉泉路,-501.8,170.6
3,131,北京,beijing,地铁1号线八通线,1号线八通线,0,23,0,-498.9,139.1,...,2025-09-28,1312509283,1,1号线八通线,4,131_001_004,131_001_005,五棵松,-453.8,170.6
4,131,北京,beijing,地铁1号线八通线,1号线八通线,0,23,0,-498.9,139.1,...,2025-09-28,1312509284,1,1号线八通线,5,131_001_005,131_001_006,万寿路,-409.2,170.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13286,51314,马德里,madeli,ML4,ML4,0,16,0,-110,1600,...,2025-09-28,51314250928440,17,ML4,12,51314_017_012,51314_017_013,Poligono Industrial Ciudad de Parla,-261.4,1434.8
13287,51314,马德里,madeli,ML4,ML4,0,16,0,-110,1600,...,2025-09-28,51314250928443,17,ML4,13,51314_017_013,51314_017_014,Jaime I Sur,-320,1494.8
13288,51314,马德里,madeli,ML4,ML4,0,16,0,-110,1600,...,2025-09-28,51314250928447,17,ML4,14,51314_017_014,51314_017_015,Estrella Polar Sur,-320,1556.1
13289,51314,马德里,madeli,ML4,ML4,0,16,0,-110,1600,...,2025-09-28,51314250928448,17,ML4,15,51314_017_015,51314_017_016,Venus Sur,-320,1616.8


In [ ]:
with open("subway_line_to_fix.json", "w", encoding="utf-8") as f:
    json.dump(df_line_main_filtered_dict, f, ensure_ascii=False, indent=4)